In [24]:
! uv add pandas pyarrow google-cloud-storage
import os
import requests
import pandas as pd
from google.cloud import storage

Resolved 136 packages in 4ms
Audited 123 packages in 2ms


In [25]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/benzenesea/Documents/GCP Keys/abiding-bongo-485522-a4-a26c7b732e16.json"

In [26]:
os.environ["GCP_GCS_BUCKET"] = "tyler_taxi_bucket"
BUCKET = os.environ["GCP_GCS_BUCKET"]

In [27]:
# services = ['fhv','green','yellow']
init_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/'

In [28]:
def upload_to_gcs(bucket, object_name, local_file):
    """
    Ref: https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
    """
    # # WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload speed.
    # # (Ref: https://github.com/googleapis/python-storage/issues/74)
    # storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    # storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB

    client = storage.Client()
    bucket = client.bucket(bucket)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)

In [29]:
def web_to_gcs(year, service):
    for i in range(12):
        
        # sets the month part of the file_name string
        month = '0'+str(i+1)
        month = month[-2:]

        # csv file_name
        file_name = f"{service}_tripdata_{year}-{month}.csv.gz"

        # download it using requests via a pandas df
        request_url = f"{init_url}{service}/{file_name}"
        r = requests.get(request_url)
        open(file_name, 'wb').write(r.content)
        print(f"Local: {file_name}")

        # read it back into a parquet file
        df = pd.read_csv(file_name, compression='gzip')
        file_name = file_name.replace('.csv.gz', '.parquet')
        df.to_parquet(file_name, engine='pyarrow')
        print(f"Parquet: {file_name}")

        # upload it to gcs 
        upload_to_gcs(BUCKET, f"{service}/{file_name}", file_name)
        print(f"GCS: {service}/{file_name}")

In [30]:
# web_to_gcs('2019', 'green')

In [31]:
# web_to_gcs('2020', 'green')

In [32]:
# web_to_gcs('2019', 'yellow')

In [33]:
# web_to_gcs('2020', 'yellow')